In [2]:
from datasets import load_dataset, DatasetDict, load_from_disk

In [2]:
# Specify the folder where the dataset was saved
cache_dir = "./datasets"

# Load the dataset from the saved folder
dataset = load_dataset("b-mc2/sql-create-context", cache_dir=cache_dir)

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 78577
    })
})

In [4]:
train_data = dataset['train']
train_data[0]

{'answer': 'SELECT COUNT(*) FROM head WHERE age > 56',
 'question': 'How many heads of the departments are older than 56 ?',
 'context': 'CREATE TABLE head (age INTEGER)'}

In [5]:
train_testval_split = dataset["train"].train_test_split(test_size=0.2, seed=42)

# Further split the test+val set into validation and test (e.g., 50% of 20% = 10% each)
test_val_split = train_testval_split["test"].train_test_split(test_size=0.5, seed=42)

# Combine splits into a DatasetDict
split_dataset = DatasetDict({
    "train": train_testval_split["train"],
    "val": test_val_split["train"],
    "test": test_val_split["test"],
})

In [6]:
split_dataset

DatasetDict({
    train: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 62861
    })
    val: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 7858
    })
    test: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 7858
    })
})

In [7]:
save_path = "./datasets/split"
split_dataset.save_to_disk(save_path)

Saving the dataset (0/1 shards):   0%|          | 0/62861 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/7858 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/7858 [00:00<?, ? examples/s]

In [3]:
split_dataset = load_from_disk("./datasets/sql-create-context-split")

In [4]:
split_dataset

DatasetDict({
    train: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 62861
    })
    val: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 7858
    })
    test: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 7858
    })
})